[1] 데이터세트 불러오기

In [4]:
import pandas as pd
from Korpora import Korpora

In [5]:
corpus = Korpora.load('nsmc')
corpusDF = pd.DataFrame(corpus.test)

train = corpusDF.sample(frac=0.9, random_state=42)
test = corpusDF.drop(train.index)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-17\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [8]:
print(train.head(5).to_markdown())
print('Training Data Size : ', len(train))
print('Testing Data Size :', len(test))

NSMCKorpus
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

Attributes
----------
NSMC.train: size=150000
  - NSMC.train.texts : list[str]
  - NSMC.train.labels : list[int]
NSMC.test: size=50000
  - NSMC.test.texts : list[str]
  - NSMC.test.labels : list[int]



[2] 데이터 토큰화 및 단어 사전 구축

In [7]:
from konlpy.tag import Okt
from collections import Counter

In [1]:
# 단어 사전 구축 함수 생성
def build_vocab(corpus, n_vocab, special_tokens) :
    counter = Counter()
    for tokens in corpus :                                  ## 각 토큰이 몇개인지 세줌
        counter.update(tokens)
    vocab = special_tokens                                  ## 최종 결과물인 vocab에 미리 <pad>,<unk> 넣어놓기 위해
    for token, count in counter.most_common(n_vocab) :      ## 빈도수가 높은 토큰들을 추가해줌 5000개로 설정할 n_vocab보다 토큰 수가 많으면 최빈 토큰만 넣기 위해
        vocab.append(token)
    return vocab

{0: '<pad>', 1: '<unk>', 2: '.', 3: '이', 4: '영화', 5: '의', 6: '..', 7: '가', 8: '에', 9: '...', 10: '을', 11: '도', 12: '들', 13: ',', 14: '는', 15: '를', 16: '은', 17: '?', 18: '너무', 19: '한', 20: '다', 21: '정말', 22: '만', 23: '진짜', 24: '적', 25: '!', 26: '로', 27: '점', 28: '으로', 29: '에서', 30: '연기', 31: '평점', 32: '과', 33: '것', 34: '~', 35: '최고', 36: '내', 37: '그', 38: '나', 39: '안', 40: '잘', 41: '와', 42: '인', 43: '생각', 44: '게', 45: '못', 46: '이런', 47: '왜', 48: '....', 49: '스토리', 50: '드라마', 51: '사람', 52: '이다', 53: '감동', 54: '1', 55: '보고', 56: '하는', 57: '때', 58: '더', 59: '하고', 60: '고', 61: '아', 62: '말', 63: '감독', 64: 'ㅋㅋ', 65: '그냥', 66: '배우', 67: '내용', 68: '거', 69: '중', 70: '재미', 71: '까지', 72: '본', 73: '보다', 74: '요', 75: '!!', 76: '없는', 77: '좀', 78: '뭐', 79: '시간', 80: '지', 81: '수', 82: '쓰레기', 83: '사랑', 84: '봤는데', 85: '볼', 86: '네', 87: '작품', 88: '다시', 89: '하나', 90: '10', 91: '없다', 92: '할', 93: '이건', 94: '마지막', 95: '2', 96: '저', 97: '같은', 98: '정도', 99: 'ㅠㅠ', 100: '있는', 101: '완전', 102: '좋은', 103: 'ㅋ', 104:

[3] 정수 인코딩 및 패딩

In [13]:
import numpy as np

In [21]:
def pad_sequences(sequences, max_length, pad_value) :
    result = list()
    for sequence in sequences :
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence = sequence + [pad_value] * pad_length
        result.append(padded_sequence)
    return np.array(result)

unk_id = token_to_id['<unk>']                                  ## token_to_id 에서 <unk>에 해당하는 인덱스 번호를 unk_id에 저장

## train_tokens에서 review를 가져오고 가져온 review를 token으로 나눔
## 나누어진 token을 token_to_id에 있는지 비교하고 있다면 인덱스를 반환 없다면 unk_id를 반환
train_ids =[
    [token_to_id.get(token,unk_id) for token in review] for review in train_tokens
]
test_ids =[
    [token_to_id.get(token,unk_id) for token in review] for review in test_tokens
]

max_length = 32
pad_id = token_to_id['<pad>']
train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

print(train_ids[:3])
print(test_ids[:3])

[[ 223 1716   10 4036 2095  193  755    4    2 2330 1031  220   26   13
  4839    1    1    1    2    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [3297  996    5  201    2   63    5 3158   16  801   37 1985    8    9
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   1   33   76  308    2    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]
[[3307    5 1997  456    8    1 1013 3906    5    1    1   13  223   51
     3    1 4684    6    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [ 141 2802   31 1406 2617    1    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   1 1404    1    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    

In [23]:
for review in train_tokens :
    # print(review)
    for token in review :
        print(token)
        break


모든
무한
신날
잔잔
오랜
영화
재밌다
음
"""
이
여운
쓰레기
솔직히
북한
올
난
픽업아티스트
별루
이
이
꺄
너희
이딴
장난
K
신난다
아무
유주
일본
후반
역시
발연기
ㅋ
생각
엄마
영웅
재밌는
완전
유지태
송은채
재미
이런게
란마
나머지
말
트루
재난영화
영어
기억
나도
책
진짜
클레멘타인
진심
놰
줄리아
아버지
런타임
평점
4
진짜
안타깝네
KTX
완
금일
정말
실제
성룡
최수종
필리
한심하다
재밌다
???
설정
어떤
이
ㅗ
드림웍스
여운
가면
제목
이건
우물
알라
와
평점
그냥
정말
나
실사
기대
bad
표적
감동
Hffufguo
나로
참
아무런
세번
악녀
4
멋진
통찰
ㅜㅠ
맛
영화
영
스릴러
모든
2015년
이
별
예쁘게
내
완전
저녁
철학
최고
진입
맥
소재
정말
예술
개인
왕조현
마음
쓰
영화
장혁
이
진짜
감동
지루하지는
2.6
이런거
너무
최고
은메달
우
너무
이런
B
너무
앗
TV
엄마
멍
코난
성질죽이기
마치
출연
최고
데이빗
무섭지도
TV
이
너무
미
좋은
좀
정말
박보영
늑대인간
어릴
배트맨
영화
재밌습니다
난
미친듯이
한마디
이렇게
별로
누가
내
매우
살아가는데
아
더티
관객
빨갱이
엘렌
감성
영화
대박
원작
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
와인
가슴
장
이스라엘
1
비디오
메르스
이
유치
옛날
중국
발
내
지루하면서
본지
엉뚱한
스포츠
어제
하하
오늘
정말
그저
스토리
포스터
멋있는
내
배갈
애니메이션
정말
악마
정말
굳
액션
롯소
정말
좋았음
주
둘
괜찮은
영화계
안읏기
좀
고증
정말
정말
중국인
강은비
밑
남주
꼭
이영화
다시
훌륭해요
바그다드
여성
뮤지컬
산만하고
여러분
영화
왕정
도구
오오
오랜
평론가
나
8.5
기대
이미
볼려고
이
내용
재미없어
스탤론
눈물
말
가볍게
주인공
이런
재미
당시
아무리
막장
뒷
즐거움
방금
조연
결말
빵빵
정말
히치콕
오버
스토리
B
초
영화
결방
메세지
정말
아
볼드윈
별로
요즘
솔직히
너무
신소율
너무
8
라이어
잠
이
2007년
당시
전지현
독특하긴
메세지
은주
재미있었어요
노래
정치
다른


[4] 데이터로더 적용

In [24]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [25]:
# 정수 인코딩 된 단어 tensor화
train_ids = torch.tensor(train_ids)             ## TensorDataset 클래스에 적용을 위해 tensor 형태로 변환
test_ids = torch.tensor(test_ids)               ## TensorDataset 클래스에 적용을 위해 tensor 형태로 변환

train_labels = torch.tensor(train.label.values, dtype=torch.float32)
test_labels = torch.tensor(test.label.values, dtype=torch.float32)

train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

[5] 문장 분류 모델

In [27]:
from torch import nn

In [31]:
class SentenceClassifier(nn.Module) :
    def __init__(
            self, n_vocab,
            hidden_dim,
            embedding_dim,
            n_layers,
            dropout=0.5,
            bidirectional=True,
            model_type='lstm'
    ) :
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=embedding_dim,
            padding_idx=0
        )
        if model_type == 'rnn' :
            self.model = nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            
            )
        elif model_type == 'lstm' :
            self.model = nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        if bidirectional :
            self.classifier = nn.Linear(hidden_dim * 2,1)
        else :
            self.classifier = nn.Linear(hidden_dim,1)
        self.dropout = nn.Dropout(dropout)

    def forward(self,inputs) :
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:, -1, :]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

[6] 손실 함수와 최적화 함수 정의

In [32]:
from torch import optim

In [33]:
n_vocab = len(token_to_id)
hidden_dim = 64
embedding_dim = 128
n_layers = 2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
classifier = SentenceClassifier(n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

[7] 모델 학습 및 테스트

In [34]:
def train(model, datasets, criterion, optimizer, device, interval) :
    model.train()
    losses = list()

    for step, (input_ids, labels) in enumerate(datasets) :
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0 :
            print(f'Train Loss{step} : {np.mean(losses)}')

def test(model, datasets, criterion, device) :
    model.eval()
    losses = list()
    corrects = list()

    for step, (input_ids, labels) in enumerate(datasets) :
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits)>.5
        corrects.extend(
            torch.eq(yhat, labels).cpu().tolist()
        )
    print(f'Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}')

epochs = 5
interval = 500

for epoch in range(epochs) :
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

Train Loss0 : 0.6934361457824707
Train Loss500 : 0.6937786169394762
Train Loss1000 : 0.6924088572884177
Train Loss1500 : 0.6796227180108001
Train Loss2000 : 0.6700299053356565
Train Loss2500 : 0.6587827641908668
Val Loss : 0.6020523107851656, Val Accuracy : 0.6792
Train Loss0 : 0.641751229763031
Train Loss500 : 0.5860450386168238
Train Loss1000 : 0.5881767242878944
Train Loss1500 : 0.582901229050698


KeyboardInterrupt: 

[8] 학습된 모델로부터 임베딩 추출

In [36]:
token_to_embedding = dict()
embedding_matrix = classifier.embedding.weight.detach().cpu().numpy()

for word, emb in zip(vocab, embedding_matrix) :
    token_to_embedding[word] = emb

token = vocab[1000]
print(token, token_to_embedding[token])

보고싶다 [ 0.3035229  -0.20412561  0.72497284  0.43052006  1.6421552  -0.28011256
  0.93324995 -2.5683315   0.03754863  0.01726936 -0.8054113   0.84207344
 -0.45113763 -0.6464346   0.6649948   0.50124425  0.9492742  -0.48965633
 -0.37522852  0.2211374  -0.72932357 -0.81544507 -0.6570944   0.35773778
  0.31472751 -2.307037   -0.6875542   1.3903974  -0.1860949  -0.54134226
 -1.2140971  -1.0657109  -0.0183298   0.210855    0.45022374 -0.9865141
 -1.6224887   0.2828206   0.21798448  0.7813731  -1.6695299   0.48136464
  2.5515826  -0.48066685 -1.3701388   3.2323978  -1.7754999  -0.5495068
 -0.06148681 -1.2976153   1.4957893   0.0767865   0.8009786  -2.1286495
  0.35271132  0.7928823  -3.486056   -0.8931781  -0.4109238  -1.8741462
 -0.253732    1.3664804   0.09191649 -1.2021407  -1.2569443   1.595602
  1.136919   -0.33334935  0.42111844 -0.37205103 -0.12402283  1.3731648
  0.4481775  -0.9243716  -0.5220712  -1.8616089   0.2619485   0.55708694
  0.9251934   2.2911127   1.8521615   0.28074616 -1.0

[토큰 만들기]

In [46]:
import pandas as pd
from Korpora import Korpora
from konlpy.tag import Okt

corpus = Korpora.load('nsmc')
corpus = pd.DataFrame(corpus.test)

tokenizer = Okt()
tokens = [tokenizer.morphs(review) for review in corpus.text]
print(tokens[:3])


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-17\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

[Word2vec 모델 학습]

In [55]:
from gensim.models import Word2Vec

In [57]:
word2Vec = Word2Vec(
    sentences=tokens,
    vector_size=128,
    window=5,
    min_count=1,
    sg=1,
    epochs=3,
    max_final_vocab=10000
)

word2Vec.save('../models/word2Vec.model')

[9] 사전 학습된 모델로 임베딩 계층 초기화

In [61]:

word2Vec = Word2Vec.load('../models/word2Vec.model')
init_embeddings = np.zeros((n_vocab, embedding_dim))

for index, token in id_to_token.items() :
    if token not in ['<pad>', '<unk>'] :
        init_embeddings[index] = word2Vec.wv[token]

embedding_layer = nn.Embedding.from_pretrained(
    torch.tensor(init_embeddings, dtype = torch.float32)
)

[10] 사전 학습된 임배딩 계층 적용

In [4]:
import torch.nn as nn
class SentenceClassifier(nn.Module) :
    def __init__(self, pretrained_embedding=None) :
        if pretrained_embedding is not None :
            self.embedding = nn.Embedding.from_pretrained(
                torch.tensor(pretrained_embedding, dtype=torch.float32)
            )
        else :
            self.embedding = nn.Embedding(
                num_embeddings=n_vocab,
                embedding_dim=embedding_dim,
                padding_idx=0
            )


[11] 사전 학습된 임베딩을 사용한 모델 학습

In [5]:
classifier = SentenceClassifier(
    n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim,
    n_layers=n_layers, pretrained_embedding=init_embeddings).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

epochs = 5
interval = 500

for epoch in range(epochs) :
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

NameError: name 'n_vocab' is not defined